# Constrained Optimization: Equality Constraints

November 2024

Joel Dieguez (niub17087652) and Clàudia Valverde (niub20441186)

**Abstract**

This practical focuses on constrained optimization by equality constraints. The method shown here consists of turning a constrained optimization problem into an unconstrained optimization problem using Lagrange multipliers.

In [ ]:
# importing necessary packages
import numpy as np
from scipy.linalg import solve

## Proposed Experiments

### 1. One simple way to proceed is to take $\alpha^k = 1$ and iteratively update the current point to obtain the next. This is a simple way to proceed that is proposed to perform first. The stopping condition should be performed over $\nabla_x L$. Test this approach and check if it works using the starting point proposed in the example

**Answer**

We want to minimize a function $𝑓(𝑥)$, subject to equality constraints $ℎ(𝑥)=0$. This can be reformulated into a Lagrangian:
$$L(x,λ)=f(x)−λh(x)$$

Using the KKT conditions, the problem reduces to solving the system of equations:



*   $\nabla_x L(x, \lambda) = \nabla f(x) - \lambda \nabla h(x) = 0$

*   $h(x) = 0$


Here, 𝑥 and 𝜆 are the variables, and we solve iteratively using a simple gradient-based approach.

Let’s solve the example provided using Sequential Quadratic Programming (SQP).
The optimization problem is:

$$
\text{minimize } f(x_1, x_2) = e^{3x_1} + e^{-4x_2}
$$

$$
\text{subject to } h(x_1, x_2) = x_1^2 + x_2^2 - 1 = 0
$$



In [ ]:
# Objective function and its gradient
def f(x0,y0):
    return np.exp(3 * x0) + np.exp(-4 * y0)

def grad_f(x0, y0):
    return np.array([3 * np.exp(3 * x0), -4 * np.exp(-4 * y0)])

In [ ]:
# Constraint function and its gradient
def h(x0, y0):
    # Constraint function
    return x0**2 + y0**2 - 1

def grad_h(x0, y0):
    # Gradient of the constraint
    return np.array([2 * x0, 2 * y0])

In [ ]:
# Hessians
def hessian_f(x0, y0):
    return np.array([[9 * np.exp(3 * x0), 0],
                     [0, 16 * np.exp(-4 * y0)]])

def hessian_h():
    return np.array([[2, 0], [0, 2]])

In [ ]:
def lagran_gradx(x0, y0, lambda_0):
    # Gradient of the Lagrangian with respect to x and y
    return grad_f(x0, y0) - lambda_0 * grad_h(x0, y0)

def lagran_hessianx(x0, y0, lambda_0):
    # Hessian of the Lagrangian
    return hessian_f(x0, y0) - lambda_0 * np.array([[2, 0], [0, 2]])

In [ ]:
# Sequential Quadratic Programming with Newton's Method
def Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol):
    for i in range(max_iter):
        # Build the matrix A to solve Ax=b
        A = np.zeros((3, 3))
        hessian = lagran_hessianx(x0, y0, lambda_0)
        gradient_h = grad_h(x0, y0)

        # Fill matrix A
        A[:2, :2] = hessian  # Top-left: Hessian of Lagrangian
        A[2, :2] = -gradient_h  # Bottom-left: -grad_h
        A[:2, 2] = -gradient_h  # Top-right: -grad_h

        # Build the vector b
        b = np.zeros(3)
        b[:2] = -lagran_gradx(x0, y0, lambda_0)  # First two entries: -grad_x L
        b[2] = h(x0, y0)  # Last entry: constraint value

        # Solve the system of equations A * delta = b
        try:
            delta = np.linalg.solve(A, b)
        except np.linalg.LinAlgError:
            print("Matrix is singular or ill-conditioned. Stopping iterations.")
            break

        # Update variables
        x0 += alpha * delta[0]
        y0 += alpha * delta[1]
        lambda_0 += alpha * delta[2]

        # Check for convergence
        if np.linalg.norm(lagran_gradx(x0, y0, lambda_0)) < tol:
            return x0, y0, lambda_0, i

    return x0, y0, lambda_0, i


In [ ]:
tol = 1e-1  # Tolerance for stopping criterion

# Run the algorithm
x_opt, y_opt, lambda_opt, iterations = Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol)

# Results
print('Iterations:', iterations)
print("Optimal x = {:.4f}".format(x_opt))
print("Optimal y = {:.4f}".format(y_opt))
print("Optimal lambda = {:.4f}".format(lambda_opt))

Iterations: 1
Optimal x = -0.7486
Optimal y = 0.6661
Optimal lambda = -0.2161


In [ ]:
tol = 1e-3  # Tolerance for stopping criterion

# Run the algorithm
x_opt, y_opt, lambda_opt, iterations = Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol)

# Results
print('Iterations:', iterations)
print("Optimal x = {:.4f}".format(x_opt))
print("Optimal y = {:.4f}".format(y_opt))
print("Optimal lambda = {:.4f}".format(lambda_opt))

Iterations: 2
Optimal x = -0.7483
Optimal y = 0.6633
Optimal lambda = -0.2123


In [ ]:
# Initial values
x0, y0 = -1.0, 1.0  # Initial guess for x and y
lambda_0 = -1.0  # Initial guess for Lagrange multiplier
alpha = 1.0  # Step size
max_iter = 100  # Maximum number of iterations
tol = 1e-6  # Tolerance for stopping criterion

# Run the algorithm
x_opt, y_opt, lambda_opt, iterations = Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol)

# Results
print('Iterations:', iterations)
print("Optimal x = {:.4f}".format(x_opt))
print("Optimal y = {:.4f}".format(y_opt))
print("Optimal lambda = {:.4f}".format(lambda_opt))

Iterations: 3
Optimal x = -0.7483
Optimal y = 0.6633
Optimal lambda = -0.2123


**Answer**

As also seen in the .pdf, the solution of this optimization problem is $(x^*, y^*) = (-0.7483, 0.6633)$ and $\lambda^* = (-2.123)$

We have tested it with different tolerances; with $1e-3$ with only 2 iterations we already reach the optimal solution. With tolerance 0.1 the convergance is reached in 1 iteration but the optimal solution is not exact. On the other hand, with tolerance 1e-6 the exact solution is found but it takes 3 iterations to reach it.

### 2. This basic iteration also has drawbacks, leading to a number of vital questions. It is a Newtonlike iteration, and thus may diverge from poor starting points. In our example we have started from a point that is near to the optimal solution. Try to perform some experiments with starting points that are farther away of the optimal solution.

In [ ]:
# Define the parameters
starting_points = [(-1.0, 1.0), (0.0, 1.0), (0.5, 0.5), (-0.9, 0.9), (4,4), (-9, 9), (20, 10), (-50, 20), (50, 100), (100, 90),]  # List of (x0, y0) starting points
lambda_0 = -1.0  # Initial guess for Lagrange multiplier
alpha = 1.0  # Step size
max_iter = 100  # Maximum number of iterations
tol = 1e-6  # Tolerance for stopping criterion

In [ ]:
# hardcoding the starting points
for idx, (x0, y0) in enumerate(starting_points):
      print(f"\nStarting Point {idx + 1}: x0 = {x0}, y0 = {y0}, lambda_0 = {lambda_0}")
      x_opt, y_opt, lambda_opt, iterations = Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol)
      print(f"Converged to: x = ({x_opt:.3f}, {y_opt:.3f}), lambda = {lambda_opt:.3f} in {iterations} iterations")


Starting Point 1: x0 = -1.0, y0 = 1.0, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 3 iterations

Starting Point 2: x0 = 0.0, y0 = 1.0, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 5 iterations

Starting Point 3: x0 = 0.5, y0 = 0.5, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 6 iterations

Starting Point 4: x0 = -0.9, y0 = 0.9, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 3 iterations

Starting Point 5: x0 = 4, y0 = 4, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 22 iterations

Starting Point 6: x0 = -9, y0 = 9, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 7 iterations

Starting Point 7: x0 = 20, y0 = 10, lambda_0 = -1.0
Converged to: x = (0.995, -0.100), lambda = 29.828 in 13 iterations

Starting Point 8: x0 = -50, y0 = 20, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 10 iterations

Starting Point 9: x0 = 50, y

<ipython-input-58-9661c4f97b00>:6: RuntimeWarning: overflow encountered in exp
  return np.array([3 * np.exp(3 * x0), -4 * np.exp(-4 * y0)])
<ipython-input-60-3a6e6144ed9f>:4: RuntimeWarning: overflow encountered in exp
  [0, 16 * np.exp(-4 * y0)]])


In [ ]:
# with random starting points

for idx in range(0, 10):
      starting_point = np.random.uniform(low=-100, high=100, size = (2,))
      x0 = starting_point[0]
      y0 = starting_point[1]
      print(f"\nStarting Point {idx + 1}: x0 = {x0}, y0 = {y0}, lambda_0 = {lambda_0}")
      x_opt, y_opt, lambda_opt, iterations = Newton_algorithm(x0, y0, lambda_0, alpha, max_iter, tol)
      print(f"Converged to: x = ({x_opt:.3f}, {y_opt:.3f}), lambda = {lambda_opt:.3f} in {iterations} iterations")


Starting Point 1: x0 = -68.81120809591097, y0 = -70.86127807483052, lambda_0 = -1.0
Converged to: x = (nan, nan), lambda = nan in 99 iterations

Starting Point 2: x0 = 95.1934399991996, y0 = 30.76158379249611, lambda_0 = -1.0
Converged to: x = (nan, nan), lambda = nan in 99 iterations

Starting Point 3: x0 = -92.77761818085047, y0 = -58.438721767097924, lambda_0 = -1.0
Converged to: x = (nan, nan), lambda = nan in 99 iterations

Starting Point 4: x0 = -34.885601124500326, y0 = 26.03955484287343, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 9 iterations

Starting Point 5: x0 = -66.31257746942586, y0 = -57.546368235343095, lambda_0 = -1.0
Converged to: x = (nan, nan), lambda = nan in 99 iterations

Starting Point 6: x0 = 48.64535569575099, y0 = -28.65391750228929, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663), lambda = -0.212 in 38 iterations

Starting Point 7: x0 = -27.14253945203629, y0 = 3.279978503692547, lambda_0 = -1.0
Converged to: x = (-0.748, 0.663)

<ipython-input-58-9661c4f97b00>:6: RuntimeWarning: overflow encountered in exp
  return np.array([3 * np.exp(3 * x0), -4 * np.exp(-4 * y0)])
<ipython-input-60-3a6e6144ed9f>:4: RuntimeWarning: overflow encountered in exp
  [0, 16 * np.exp(-4 * y0)]])
<ipython-input-60-3a6e6144ed9f>:3: RuntimeWarning: overflow encountered in exp
  return np.array([[9 * np.exp(3 * x0), 0],


**Answer:**

For experimental propuses, we have tried the algorithm with some hard-coded random starting points to see the behaviour as we incrementaly distenciate ourselves from the original given starting point (-1,1). On the other hand, we have also tried with 10 random starting points using the np.random function to get combination of points that we might not have though of while creating the hard-coded ones.

We obeserve that as the distance from the original starting point increases, the more iterations are needed to reach the convergance, in some cases the convergance is not reached in less than 100 iterations while in others we even get an overflow error indicating the values of the points are too high to compute the hessian.

Overall this indicates that the Newton Algorithm works best in a local way, as for further points the method does not work.

### 3. One way to find the optimal solution from points that are far away of the optimal solution is to start the optimization with another function that allows us to find an approximation to the solution we are looking for. Once an approximate solution is found, we can apply the Newton-based technique we have presented previously to find the optimal solution.

### The function that allows us to find an approximation to the solution we are looking for is called, in this context, the merit function. Usually, a merit function is the sum of terms that include the objective function and the amount of infeasibility of the constraints. One example of a merit function for the problem we are treating is the quadratic penalty function (i.e. constraints are penalized quadratically)

$
M(x_1, x_2) = f(x_1, x_2) + ρh(x_1, x_2)^2
$

### where $ρ$ is some positive number. The greater the value of $ρ$, the greater the penalty for infeasibility. The difficulty arises in defining a proper merit function for a particular equality constrained problem. Here we propose you to take $ρ = 10$ (thus, we penalize a lot the constraint) and perform a classical gradient descent (with backtraking if you want) to find and approximation to the solution we are looking for. Observe if you arrive near to the optimal solution of the problem.

### Take into account that you may have numerical problems with the gradient. A simple way to deal with it is to normalize the gradient at each iteration, $\nablaΜ(x)/ ||\nabla M(x)||$, and use this normalized gradient as search direction.

In [ ]:
# Merit function and its gradient
def merit_function(x, rho):
    return f(x[0], x[1]) + rho * h(x[0], x[1])**2

def grad_merit_function(x, rho):
    grad_f_val = grad_f(x[0], x[1])
    grad_h_val = grad_h(x[0], x[1])
    h_val = h(x[0], x[1])
    return grad_f_val + 2 * rho * h_val * grad_h_val

In [ ]:
# Gradient descent for merit function
def gradient_descent_merit(x0, rho, alpha, max_iter, tol):
    x = np.array(x0, dtype=float)

    for i in range(max_iter):
        grad_M = grad_merit_function(x, rho)
        grad_norm = np.linalg.norm(grad_M)

        # Normalized gradient descent step
        x_new = x - alpha * grad_M / grad_norm

        # Backtracking line search
        while merit_function(x_new, rho) >= merit_function(x, rho):
            alpha *= 0.5
            x_new = x - alpha * grad_M

        if abs(merit_function(x_new, rho) - merit_function(x, rho)) < tol or  np.linalg.norm(grad_M/grad_norm) < tol:
            print(f"Converged to: x = ({x[0]:.3f}, {x[1]:.3f}) in {i} iterations")
            return x_new

        if i % 10 == 0:  # Optional logging every 10 iterations
            print(f"Iteration {i}: x = ({x[0]:.3f}, {x[1]:.3f}), Gradient Norm = {grad_norm:.3e}")

        else:
          x = x_new

    print(f"Did not converge after {max_iter} iterations. Last x = ({x[0]:.3f}, {x[1]:.3f})")
    return x

In [ ]:
# Test gradient descent with merit function
rho = 10
alpha = 1
max_iter = 1000
tol = 1e-3

x0 = [-0.5, 10]  # Random Starting point
approx_solution = gradient_descent_merit(x0, rho, alpha, max_iter, tol)

Iteration 0: x = (-0.500, 10.000), Gradient Norm = 3.975e+04
Iteration 10: x = (-0.061, 1.011), Gradient Norm = 2.618e+00
Iteration 20: x = (-0.422, 0.913), Gradient Norm = 7.244e-01
Iteration 30: x = (-0.545, 0.845), Gradient Norm = 4.178e-01
Converged to: x = (-0.623, 0.795) in 37 iterations


In [ ]:
# Test gradient descent with merit function
rho = 10
alpha = 1
max_iter = 1000
tol = 1e-3

x0 = [-1, 1]  # Change starting point
approx_solution = gradient_descent_merit(x0, rho, alpha, max_iter, tol)

Iteration 0: x = (-1.000, 1.000), Gradient Norm = 5.641e+01
Converged to: x = (-0.717, 0.710) in 7 iterations


**Answer**

We can see that we are reaching close to the optimal value by changing the starting point and getting help from the Merit function...

### 4. As previously commented, the minimizers of the merit function $M(x_1, x_2)$ do not necessarily have to coincide with the minimizers of the constrained problem. Thus, once we “sufficiently” approach the optimal solution we may use the Newton method (with $α = 1$) to find the solution to the problem.

### Therefore the algorithm consists in starting with the Merit function to obtain an approximation to the optimal point we are looking for. Once an approximation to the solution is found, use the Newton-based method to find the optimal solution. Check if you are able to find the optimal solution to the problem.

In [ ]:
print(approx_solution)

[-0.71570856  0.70752538]


In [ ]:
solution = Newton_algorithm(approx_solution[0], approx_solution[1], -1, 1, 100, 1e-3)

In [ ]:
x0, y0, lambda0, iterations = solution

print('Iterations to reach optimal solution:', iterations)
print('Optimal Solution:', x0, ',', y0)
print('Optimal lambda', lambda0)

Iterations to reach optimal solution: 2
Optimal Solution: -0.7483354358209479 , 0.6633206217919233
Optimal lambda -0.21232445155672172


**Answer**

Starting with the approx solution found above (with the merit function), we have been able to use Newton to obtain the exact same solution as in the pdf. The method works.